<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Any_to_English_AI_Video_Subtitle_Captioning_App_with_OpenAI_Whisper_App_Full_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/openai/whisper.git -q

     |████████████████████████████████| 4.9 MB 34.7 MB/s 
     |████████████████████████████████| 120 kB 75.0 MB/s 
     |████████████████████████████████| 6.6 MB 85.7 MB/s 


In [2]:
! pip install gradio -q

     |████████████████████████████████| 5.3 MB 32.6 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 92.5 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 68.8 MB/s 
     |████████████████████████████████| 270 kB 82.1 MB/s 
     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 67.3 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 80 kB 10.3 MB/s 
     |████████████████████████████████| 68 kB 7.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 594 kB 93.0 MB/s 
     |████████████████████████████████| 4.0 MB 82.8 MB/s 
     |████████████████████████████████| 856 kB 78.3 MB/s 


In [6]:
! pip install pytube -q

     |████████████████████████████████| 56 kB 4.3 MB/s 


## Download the YouTube Video

In [1]:
# importing the module  
from pytube import YouTube  
yt = YouTube('https://youtu.be/oQdxL_WW3aE')  
print(yt.title)  

Shamshera | Date Announcement Teaser | Ranbir Kapoor | Sanjay Dutt | Vaani Kapoor


In [4]:
myStream = YouTube("https://www.youtube.com/shorts/dvlMvUhH3IY").streams.first()
myStream.download()

'/content/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts.3gpp'

## Import Necessary Library

In [5]:
import gradio as gr 
import os
import sys
import subprocess

In [6]:
import whisper
model = whisper.load_model("medium")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

100%|█████████████████████████████████████| 1.42G/1.42G [00:24<00:00, 63.4MiB/s]


In [7]:
model.device

device(type='cuda', index=0)

## Save youtube video as mp3 file

In [8]:
def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"

In [9]:
input_video = "/content/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts.3gpp"
audio_file = video2mp3(input_video)

In [10]:
audio_file

'/content/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts.mp3'

## Play Audio

In [11]:
from IPython.display import Audio
Audio(audio_file,rate=22050)

In [12]:
def translate(audio):
    
    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)
    return result

In [13]:
result = translate(audio_file)

Detected language: Hindi


100%|██████████| 5475/5475 [00:29<00:00, 184.34frames/s]


In [14]:
print(result["text"])

 Father and grandfather have divided the country into two parts and today they are saying that we will join the country. To defeat a tea vendor, they have prepared a whole lot of great politics elections. They agreed but didn't get a seat in some places. But after a little struggle, when they again, and after that, a true politician who talks about Hindutva will sit on that throne, then they will understand their status when in 28-28 states, in 28-28 states, only one government will run. I am Hindutva's politician.


## Play Audio using HTML

In [15]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(input_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Write transcripts

In [16]:
from whisper.utils import write_vtt

In [17]:
output_dir = '/content/'
audio_path = audio_file.split(".")[0]

In [18]:
audio_path

'/content/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts'

In [19]:
with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
    write_vtt(result["segments"], file=vtt)

In [23]:
subtitle = audio_path + ".vtt"
output_video = audio_path + "_subtitled.mp4"

In [27]:
output_video

'/content/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts_subtitled.mp4'

In [26]:
os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")

256

## Write the converted text in English to Videoframe

In [28]:
subprocess.call(["ffmpeg", "-i", input_video , "-vf", f"subtitles={subtitle}", f"{output_video}"], 
                 stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT)

0

## Play the VIDEO WITH ENGLISH captions

In [29]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Gradio Interface

In [ ]:
import gradio as gr 
import os
import sys
import subprocess
#from moviepy.editor import VideoFileClip

import whisper
from whisper.utils import write_vtt

model = whisper.load_model("medium")

def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


def translate(input_video):

    audio_file = video2mp3(input_video)
    
    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)

    output_dir = '/content/'
    audio_path = audio_file.split(".")[0]
    print(audio_path)

    with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
      write_vtt(result["segments"], file=vtt)

    subtitle = audio_path + ".vtt"
    output_video = audio_path + "_subtitled.mp4"
    print(output_video)

    os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")
    subprocess.call(["ffmpeg", "-i", input_video , "-vf", f"subtitles={subtitle}", f"{output_video}"], 
                 stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT)

    return output_video



title = "Add Text/Caption to your YouTube Shorts - MultiLingual"

block = gr.Blocks()

with block:

    with gr.Group():
        with gr.Box(): 

           

            with gr.Row().style():
               
                inp_video = gr.Video(
                    label="Input Video",
                    type="filepath",
                    mirror_webcam = False
                )
                op_video = gr.Video()
            btn = gr.Button("Generate Subtitle Video")
        
        
        


        
        btn.click(translate, inputs=[inp_video], outputs=[op_video])
 
        gr.HTML('''
        <div class="footer">
                    <p>Model by <a href="https://github.com/openai/whisper" style="text-decoration: underline;" target="_blank">OpenAI</a> - Gradio App by <a href="https://github.com/plaban1981" style="text-decoration: underline;" target="_blank">1littlecoder</a>
                    </p>
        </div>
        ''')

block.launch(debug = True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21439.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Detected language: Hindi


100%|██████████| 5475/5475 [00:23<00:00, 235.10frames/s]


/tmp/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts61u8rwkn
/tmp/राहुल गांधी जी का सपना 😂 sanatan hindu rss bajarangdal shorts61u8rwkn_subtitled.mp4
